In [1]:
# %load symmetric_encryption.py
#!/usr/bin/env/python3

import urllib.request
from Crypto.Cipher import AES
import binascii
import base64
import random
import os
import zlib

################################################################################
# CS 284 Padding Utility Functions
################################################################################

# s is a bytearray to pad, k is blocklength
# you won't need to change the block length
def cmsc284pad(s,k=16):
  if k > 255:
    print("pkcs7pad: padding block length must be less than 256")
    return bytearray()
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s.extend([i])
  return s

# s is bytes to pad, k is blocklength
# you won't need to change the block length
def cmsc284padbytes(s,k=16):
  if k > 255:
    raise Exception("pkcs7pad: padding block length must be less than 256")
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s += chr(i).encode("utf-8")
  return s

# s is bytes to unpad, k is blocklength
# you won't need to change the block length
def cmsc284unpad(s,k=16):
  if not cmsc284checkpadding(s,k):
    print("cmsc284unpad: invalid padding")
    return b''
  n = s[len(s)-1]
  return s[:len(s)-n]

# checks padding on s and returns a boolean
# you won't need to change the block length
def cmsc284checkpadding(s,k=16):
  if(len(s) == 0):
    #print("Invalid padding: String zero length"%k) 
    return False
  if(len(s)%k != 0): 
    #print("Invalid padding: String is not multiple of %d bytes"%k) 
    return False
  n = s[len(s)-1]
  if n > k or n == 0:
    return False
  else: 
    for i in range(n):
      if s[len(s)-1-i] != (n-i):
        return False
  return True

################################################################################
# Function for querying the server
################################################################################

PPS2SERVER = "http://cryptoclass.cs.uchicago.edu/"
def make_query(task, cnetid, query):
  DEBUG = False
  if DEBUG:
    print("making a query")
    print("Task:", task)
    print("CNET ID:", cnetid)
    print("Query:", query)
  if (type(query) is bytearray) or (type(query) is bytes):
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query)) + "/"
  else:
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query.encode('utf-8'))) + "/"
  if DEBUG:
    print("Querying:", url)

  with urllib.request.urlopen(url) as response:
    raw_answer = response.read()
    answer = base64.urlsafe_b64decode(raw_answer)
    if DEBUG:
      print("Answer:", answer)
    return answer
  return None


################################################################################
# Problem 1 SOLUTION
################################################################################

def problem1(cnetid):
  return b''


################################################################################
# Problem 2 SOLUTION
################################################################################

def problem2(cnetid):
  return b''


################################################################################
# Problem 3 SOLUTION
################################################################################

def problem3(cnetid):
  return b''


################################################################################
# Problem 4 SOLUTION
################################################################################

def problem4(cnetid):
  return b''


################################################################################
# Problem 5 SOLUTION
################################################################################

def problem5(cnetid):
  return b''

################################################################################
# Problem 6 SOLUTION
################################################################################

def problem6(cnetid):
  return b''

In [2]:
from collections import Counter
import math

Clean version

In [3]:
def get_flag_len(cnetid):
  """get unpadded flag len"""
  padded_flag_len = len(make_query('three', cnetid, ''))
  flag_len = -1
  prev = -1
  for i in range(1, 17):
    curr = len(make_query('three', cnetid, bytes(i)))
    if prev == padded_flag_len and curr == padded_flag_len + 16:
      flag_len = padded_flag_len - i
      break
    prev = curr
  return flag_len

In [5]:
# sanity check, 64 - 37 = 27
get_flag_len('davidcash'), \
len(make_query('three', 'davidcash', bytes(26))), \
len(make_query('three', 'davidcash', bytes(27)))

(37, 64, 80)

In [20]:
def get_byte_map(prev_bytes):
  # should feed this function prev_bytes[:16]
  if len(prev_bytes) > 16:
    raise
  mp = {}
  for i in range(256):
    query_bytes = bytes([i]) + prev_bytes
    query = cmsc284padbytes(query_bytes)
    ctext = make_query('three', 'ruolinzheng', query)
    # always only need the first block
    mp[ctext[:16]] = query_bytes
  return mp

In [21]:
def get_block_loc(flag_len):
  """get start pos of the target block"""
  block_idx = math.ceil((flag_len - 1) / 16)
  start = 16 * block_idx
  return start

In [22]:
def get_first_query_len(flag_len):
  """get the length of the first query, [1, 16]"""
  return (16 - (flag_len - 1) % 16) % 16

In [23]:
get_block_loc(31), get_first_query_len(31), \
get_block_loc(33), get_first_query_len(33), \
get_block_loc(37), get_first_query_len(37)

(32, 2, 32, 0, 48, 12)

In [25]:
def get_flag(cnetid):
  flag_len = get_flag_len(cnetid)
  start = get_block_loc(flag_len)
  first_query_len = get_first_query_len(flag_len)
  print('flag len, start, first_query_len:',
        flag_len, start, first_query_len)
  prev_bytes = b''
  for i in range(flag_len):
    byte_mp = get_byte_map(prev_bytes[:16])
    query = bytes(first_query_len + len(prev_bytes))
    ctext = make_query('three', cnetid, query)
    new_bytes = byte_mp[ctext[start : start + 16]]
    prev_bytes = bytes([new_bytes[0]]) + prev_bytes
    print(i, prev_bytes)
  return prev_bytes

In [26]:
get_flag('ffalzon')

flag len, start, first_query_len: 34 48 15
0 b'n'
1 b'an'
2 b'man'
3 b' man'
4 b'a man'
5 b' a man'
6 b'f a man'
7 b'of a man'
8 b' of a man'
9 b'o of a man'
10 b'to of a man'
11 b'tto of a man'
12 b'etto of a man'
13 b'uetto of a man'
14 b'ouetto of a man'
15 b'houetto of a man'
16 b'lhouetto of a man'
17 b'ilhouetto of a man'
18 b'silhouetto of a man'
19 b' silhouetto of a man'
20 b'e silhouetto of a man'
21 b'le silhouetto of a man'
22 b'tle silhouetto of a man'
23 b'ttle silhouetto of a man'
24 b'ittle silhouetto of a man'
25 b'little silhouetto of a man'
26 b' little silhouetto of a man'
27 b'a little silhouetto of a man'
28 b' a little silhouetto of a man'
29 b'e a little silhouetto of a man'
30 b'ee a little silhouetto of a man'
31 b'see a little silhouetto of a man'
32 b' see a little silhouetto of a man'
33 b'I see a little silhouetto of a man'


b'I see a little silhouetto of a man'

Scratch

In [3]:
# get padded FLAG len
padded_flag_len = len(make_query('three', 'ruolinzheng', ''))

In [4]:
padded_flag_len

48

In [5]:
# determine unpadded FLAG len, [padded_flag_len - 16, padded_flag_len - 1]
len(make_query('three', 'ruolinzheng', bytes(14))), \
len(make_query('three', 'ruolinzheng', bytes(15)))

(48, 64)

In [6]:
# bytes(16) || FLAG is either padded_flag_len or padded_flag_len + 16
prev = -1
for i in range(1, 17):
  curr = len(make_query('three', 'ruolinzheng', bytes(i)))
  if prev == padded_flag_len and curr == padded_flag_len + 16:
    print('FLAG length', padded_flag_len - i)
    break
  prev = curr

FLAG length 33


In [7]:
# unpadded FLAG len 33
flag_len = 33

In [40]:
# manipulate query content to get 1 byte || 15 byte-pad mapping
mp = {}
for i in range(256):
  one_byte = bytes([i])
  query = cmsc284padbytes(one_byte)
  ctext = make_query('three', 'ruolinzheng', query)
  # len(ctext) is 64, three blocks, need first block
  mp[ctext[:16]] = one_byte

In [47]:
# manipulate len of query to push last byte of FLAG to its own block
ctext = make_query('three', 'ruolinzheng', '')
# len 48, 3 blocks, need last block
mp[ctext[-16:]] # last byte of FLAG

b' '

In [52]:
# second-to-last byte of FLAG
mp = {}
for i in range(256):
  two_bytes = bytes([i]) + b' '
  query = cmsc284padbytes(two_bytes)
  ctext = make_query('three', 'ruolinzheng', query)
  # len(ctext) is 64, three blocks, need first block
  mp[ctext[:16]] = two_bytes

In [58]:
ctext = make_query('three', 'ruolinzheng', bytes(1))
# len 48, 3 blocks, need last block
mp[ctext[-16:]] # last byte of FLAG

b'f '

In [32]:
def get_byte(prev_bytes):
  # should feed this function prev_bytes[:16]
  mp = {}
  for i in range(256):
    query_bytes = bytes([i]) + prev_bytes
    query = cmsc284padbytes(query_bytes)
    ctext = make_query('three', 'ruolinzheng', query)
    # len(ctext) is 64, three blocks, need first block
    mp[ctext[:16]] = query_bytes
  ctext = make_query('three', 'ruolinzheng', bytes(len(prev_bytes)))
  return mp[ctext[-16:]]

In [33]:
# retrieve first block
prev_bytes = b''
for i in range(15):
  prev_bytes = get_byte(prev_bytes)
  print(i, prev_bytes)

0 b' '
1 b'f '
2 b'lf '
3 b'elf '
4 b'self '
5 b'yself '
6 b'myself '
7 b' myself '
8 b'l myself '
9 b'el myself '
10 b'uel myself '
11 b'fuel myself '
12 b' fuel myself '
13 b'e fuel myself '
14 b'le fuel myself '


In [30]:
len(b'le fuel myself ') # first block

15

In [67]:
# second block
def get_byte_map(prev_bytes):
  # should feed this function prev_bytes[:16]
  mp = {}
  for i in range(256):
    query_bytes = bytes([i]) + prev_bytes
    query = cmsc284padbytes(query_bytes)
    ctext = make_query('three', 'ruolinzheng', query)
    # always only need the first block
    mp[ctext[:16]] = query_bytes
  return mp

In [64]:
prev_bytes = b'le fuel myself '
byte_mp = get_byte_map(prev_bytes)

32 80


In [48]:
ctext = make_query('three', 'ruolinzheng', bytes(len(prev_bytes)))
# now 64 bytes b/c padding, 4 blocks, need entire 3rd block
prev_bytes = byte_mp[ctext[32:48]]
prev_bytes

b'tle fuel myself '

In [52]:
byte_mp = get_byte_map(prev_bytes[:16])
ctext = make_query('three', 'ruolinzheng', bytes(len(prev_bytes)))
prev_bytes = byte_mp[ctext[32:48]]
prev_bytes

b'ttle fuel myself '

In [66]:
byte_mp = get_byte_map(prev_bytes[:16])
ctext = make_query('three', 'ruolinzheng', bytes(len(prev_bytes)))
new_bytes = byte_mp[ctext[32:48]]
new_bytes, prev_bytes # need to concatenate these two

32 80


(b'ittle fuel myself', b'ttle fuel myself ')

In [70]:
prev_bytes = bytes([new_bytes[0]]) + prev_bytes
byte_mp = get_byte_map(prev_bytes[:16])
ctext = make_query('three', 'ruolinzheng', bytes(len(prev_bytes)))
new_bytes = byte_mp[ctext[32:48]]
new_bytes, prev_bytes # need to concatenate these two

(b'little fuel mysel', b'ittle fuel myself ')

In [72]:
# try on remaining
prev_bytes = b'le fuel myself '
for i in range(15, 33):
  byte_mp = get_byte_map(prev_bytes[:16])
  ctext = make_query('three', 'ruolinzheng', bytes(len(prev_bytes)))
  new_bytes = byte_mp[ctext[32:48]] # note that this is at most 16 bytes
  prev_bytes = bytes([new_bytes[0]]) + prev_bytes
  print(i, prev_bytes)

b'tle fuel myself '
b'ttle fuel myself '
b'ittle fuel myself '
b'little fuel myself '
b' little fuel myself '
b'a little fuel myself '
b' a little fuel myself '
b'e a little fuel myself '
b'se a little fuel myself '
b'use a little fuel myself '
b' use a little fuel myself '
b'd use a little fuel myself '
b'ld use a little fuel myself '
b'uld use a little fuel myself '
b'ould use a little fuel myself '
b'could use a little fuel myself '
b' could use a little fuel myself '
b'I could use a little fuel myself '
